In [1]:
import sys
import gi
import logging

gi.require_version("GLib", "2.0")
gi.require_version("GObject", "2.0")
gi.require_version("Gst", "1.0")

from gi.repository import Gst, GLib, GObject

uri_name='rtsp://127.0.0.1:8554/test'
file='/home/fkurt/Murat/ElephantsDream.mp4'

global decoder

def cb_newpad(demux, src, sink):
    print("In cb_newpad\n")
    caps=src.get_current_caps()
    gststruct=caps.get_structure(0)
    gstname=gststruct.get_name()
    features=caps.get_features(0)

    print("gstname=",gstname)
    if(gstname.find("video")!=-1):
        print("features=",features)
        sink_pad=sink.get_static_pad("sink")
        if not src.link(sink_pad):
            sys.stderr.write("Failed to link decoder src pad to source bin ghost pad\n")

def stop_release_source(source_id):
    global decoder
    global demux
    global pipeline

    #Attempt to change status of source to be released 
    state_return = decoder.set_state(Gst.State.NULL)

    if state_return == Gst.StateChangeReturn.SUCCESS:
        print("STATE CHANGE SUCCESS\n")
        pad_name = "sink_%u" % source_id
        print(pad_name)
        #Retrieve sink pad to be released
        sinkpad = demux.get_static_pad(pad_name)
        #Send flush stop event to the sink pad, then release from the streammux
        sinkpad.send_event(Gst.Event.new_flush_stop(False))
        demux.release_request_pad(sinkpad)
        print("STATE CHANGE SUCCESS\n")
        #Remove the source bin from the pipeline
        pipeline.remove(decoder)
        source_id -= 1
        g_num_sources -= 1

    elif state_return == Gst.StateChangeReturn.FAILURE:
        print("STATE CHANGE FAILURE\n")
    
    elif state_return == Gst.StateChangeReturn.ASYNC:
        state_return = decoder.get_state(Gst.CLOCK_TIME_NONE)
        pad_name = "sink_%u" % source_id
        print(pad_name)
        sinkpad = demux.get_static_pad(pad_name)
        sinkpad.send_event(Gst.Event.new_flush_stop(False))
        demux.release_request_pad(sinkpad)
        print("STATE CHANGE ASYNC\n")
        pipeline.remove(decoder)

        nvdecoder=Gst.ElementFactory.make('nvv4l2decoder','decoder')
        pipeline.add


logging.basicConfig(level=logging.DEBUG, format="[%(name)s] [%(levelname)8s] - %(message)s")
logger = logging.getLogger(__name__)

Gst.init(None)

pipeline=Gst.Pipeline.new('test-pipeline')

source=Gst.ElementFactory.make('filesrc','source')
source.set_property('location', file)

demux=Gst.ElementFactory.make("qtdemux", "demux")
if not demux:
    sys.stderr.write(" Unable to create demux \n")
decoder=Gst.ElementFactory.make('avdec_h264','decoder')

demux.connect("pad-added", cb_newpad, decoder)

videoconvert=Gst.ElementFactory.make('videoconvert','convert')
x264enc=Gst.ElementFactory.make('x264enc','encoder')

sink=Gst.ElementFactory.make('rtspclientsink','sink')
if not sink:
    sys.stderr.write(" Unable to create udpsink")
sink.set_property("location", uri_name)

if not sink or not decoder or not videoconvert or not x264enc or not pipeline:
    logger.error('Elementlerde sıkıntı var...')
    sys.exit(1)

pipeline.add(source)
pipeline.add(demux)
pipeline.add(decoder)
pipeline.add(videoconvert)
pipeline.add(x264enc)
pipeline.add(sink)

if source.link(demux):
    logger.info('src-demux Bağlandı')
if decoder.link(videoconvert):
    logger.info('decoder-videoconvert Bağlandı')
if videoconvert.link(x264enc):
    logger.info('videoconvert-x264enc Bağlandı')
if x264enc.link(sink):
    logger.info('x264enc-sink Bağlandı')

ret=pipeline.set_state(Gst.State.PLAYING)

if ret==Gst.StateChangeReturn.FAILURE:
    logger.error("pipeline'da sıkıntı var")
    sys.exit(1)

bus = pipeline.get_bus()
msg = bus.timed_pop_filtered(
    Gst.CLOCK_TIME_NONE, 
    Gst.MessageType.ERROR | Gst.MessageType.EOS)

if msg:
    if msg.type == Gst.MessageType.ERROR:
        err, debug_info = msg.parse_error()
        logger.error(f"Error received from element {msg.src.get_name()}: {err.message}")
        logger.error(f"Debugging information: {debug_info if debug_info else 'none'}")

    elif msg.type == Gst.MessageType.EOS:
        logger.info("End-Of-Stream reached.")

    else:
        # This should not happen as we only asked for ERRORs and EOS
        logger.error("Unexpected message received.")

pipeline.set_state(Gst.State.NULL)

SyntaxError: invalid syntax (585187772.py, line 35)

In [4]:
import logging
from threading import Thread, Event

from tools.application_init import application_init

application_init()

from gi.repository import Gst, GLib
from tools.logging_pad_probe import logging_pad_probe
from tools.runner import Runner

log = logging.getLogger("main")

log.info("building pipeline")
pipeline = Gst.Pipeline.new()
caps = Gst.Caps.from_string("audio/x-raw,format=S16LE,rate=48000,channels=2")

testsrc1 = Gst.ElementFactory.make("audiotestsrc", "testsrc1")
testsrc1.set_property("is-live", True)  # (3)
testsrc1.set_property("freq", 220)
pipeline.add(testsrc1)

mixer = Gst.ElementFactory.make("audiomixer")
pipeline.add(mixer)
testsrc1.link_filtered(mixer, caps)

sink = Gst.ElementFactory.make("autoaudiosink")
pipeline.add(sink)
mixer.link_filtered(sink, caps)

testsrc1.get_static_pad("src").add_probe(
    Gst.PadProbeType.BUFFER, logging_pad_probe, "testsrc1-output")

mixer.get_static_pad("src").add_probe(
    Gst.PadProbeType.BUFFER, logging_pad_probe, "mixer-output")


def add_new_src():
    log.info("Adding testsrc2")
    testsrc2 = Gst.ElementFactory.make("audiotestsrc", "testsrc2")
    testsrc2.set_property("freq", 440)
    testsrc2.set_property("is-live", True)  # (2)

    testsrc2.get_static_pad("src").add_probe(
        Gst.PadProbeType.BUFFER, logging_pad_probe, "testsrc2-output")

    pipeline.add(testsrc2)
    testsrc2.link_filtered(mixer, caps)
    testsrc2.sync_state_with_parent()  # (4)
    log.info("Adding testsrc2 done")


stop_event = Event()


def timed_sequence():
    log.info("Starting Sequence")
    if stop_event.wait(2): return
    GLib.idle_add(add_new_src)  # (1)
    log.info("Sequence ended")


t = Thread(target=timed_sequence, name="Sequence")
t.start()

runner = Runner(pipeline)
runner.run_blocking()

stop_event.set()
t.join()

ModuleNotFoundError: No module named 'tools'